In [ ]:
!wget --header="Host: doc-10-7c-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Cookie: AUTH_gbsevnc13krov7rjgq2cj0s062jqolpo_nonce=vgdcsfp52t5m4" --header="Connection: keep-alive" "https://doc-10-7c-docs.googleusercontent.com/docs/securesc/ths62md7879q0rdb5g663rm4c96q2241/htppf4vboej968isgqfsq1vahttiph9q/1622633025000/00484516897554883881/06568700698331676125/13exfXiyiByluh1PfYK1EyZyizqxeCVG9?e=download&authuser=0&nonce=vgdcsfp52t5m4&user=06568700698331676125&hash=ui7jlmmmhg2cc1b50rc1cjsern614lfr" -c -O 'tokenization.py'

--2021-06-02 11:24:26--  https://doc-10-7c-docs.googleusercontent.com/docs/securesc/ths62md7879q0rdb5g663rm4c96q2241/htppf4vboej968isgqfsq1vahttiph9q/1622633025000/00484516897554883881/06568700698331676125/13exfXiyiByluh1PfYK1EyZyizqxeCVG9?e=download&authuser=0&nonce=vgdcsfp52t5m4&user=06568700698331676125&hash=ui7jlmmmhg2cc1b50rc1cjsern614lfr
Resolving doc-10-7c-docs.googleusercontent.com (doc-10-7c-docs.googleusercontent.com)... 142.250.141.132, 2607:f8b0:4023:c0b::84
Connecting to doc-10-7c-docs.googleusercontent.com (doc-10-7c-docs.googleusercontent.com)|142.250.141.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17318 (17K) [text/x-python]
Saving to: ‘tokenization.py’

tokenization.py     100%[===================>]  16.91K  --.-KB/s    in 0.003s  

2021-06-02 11:24:26 (6.04 MB/s) - ‘tokenization.py’ saved [17318/17318]



Import Libraries

In [ ]:
import pandas as pd
import pickle
import tokenization
from tokenization import FullTokenizer # reference :appliedai

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from tensorflow.keras.layers import  Flatten,Dropout,BatchNormalization,Conv1D, MaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy,categorical_crossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras import callbacks
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding
from sklearn.metrics  import f1_score,roc_auc_score, auc

In [ ]:
#read train and validation and test data
train_data=pd.read_csv('/content/drive/MyDrive/Meme/data/train_data.csv')
validation_data=pd.read_csv("/content/drive/MyDrive/Meme/data/validation_data.csv")
test_data=pd.read_csv('/content/drive/MyDrive/Meme/data/test_data.csv')

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Meme/data/final_dank.csv')
data.shape

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,4,5,8,9,10,11,14,16,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(76138, 68)

In [ ]:
## Loading the Pretrained Model from tensorflow HUB
tf.keras.backend.clear_session()

max_seq_length = 40

#BERT takes 3 inputs

#this is input words. Sequence of words represented as integers
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_word_ids")

#mask vector if you are padding anything
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_mask")

#segment vectors. If you are giving only one sentence for the classification, total seg vector is 0. 
#If you are giving two sentenced with [sep] token separated, first seq segment vectors are zeros and 
#second seq segment vector are 1's
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="segment_ids")

#bert layer 
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1", trainable=False)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

bert_model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=pooled_output)


In [ ]:
bert_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 40)]         0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 40)]         0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 40)]         0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [ ]:
bert_model.output

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'keras_layer')>

In [ ]:
#getting Vocab file
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

In [ ]:
#reference: applied ai
tokenizer=FullTokenizer(vocab_file,do_lower_case)

In [ ]:
test_tokens=[]
train_tokens=[]
validation_tokens=[ ]
for i in train_data['text']:
  tokens=tokenizer.tokenize(i)
  train_tokens.append(tokens[0:38])
for j in test_data['text']:
  tokens=tokenizer.tokenize(i)
  test_tokens.append(tokens[0:38])
for k in validation_data['text']:
  tokens=tokenizer.tokenize(i)
  validation_tokens.append(tokens[0:38])

In [ ]:
for index,i in enumerate(train_tokens):
    train_tokens[index]=np.array(['[CLS]',*i,'[SEP]'])
 
for index,i in enumerate(test_tokens):
    test_tokens[index]=np.array(['[CLS]',*i,'[SEP]'])  

for index,i in enumerate(test_tokens):
    test_tokens[index]=np.array(['[CLS]',*i,'[SEP]'])     

In [ ]:
print(train_tokens[9])

['[CLS]' 'every' 'time' 'the' 'pee' 'when' 'you' 'wig' '##gle' 'it' 'and'
 'wait' 'the' 'pee' 'when' 'you' 'lift' 'your' 'underwear' 'up' '[SEP]']


In [ ]:
def padding(data):
  for index,i in enumerate(data):
    sen_len=len(i)
    if sen_len<max_seq_length :
      diff=max_seq_length -sen_len
      data[index]=np.array([*i,*["[PAD]" for k in range(diff)]])
  return data    

In [ ]:
train_tokens=padding(train_tokens)
validation_tokens=padding(validation_tokens)
test_tokens=padding(test_tokens)

In [ ]:
def mask(data):
  mask_data=[]
  for i in data:
    mask_data.append(list(map(lambda x:0 if x=='[PAD]' else 1,i)))
  return mask_data


In [ ]:
train_mask=mask(train_tokens)
test_mask =mask(test_tokens)
validation_mask=mask(validation_tokens)

In [ ]:
train_segment=np.zeros((len(train_mask),max_seq_length ))
validation_segment=np.zeros((len(validation_mask),max_seq_length ))
test_segment=np.zeros((len(test_mask),max_seq_length ))
print(train_segment.shape,test_segment.shape,validation_segment.shape)

(3264, 40) (1637, 40) (1615, 40)


In [ ]:
print(train_tokens[2:5])

[array(['[CLS]', 'my', 'first', 'in', 'qu', '##aran', '##tine', 'me', 'an',
       'ex', '##tro', '##vert', 'that', 'people', 'in', 'qu', '##aran',
       '##tine', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]'], dtype='<U6'), array(['[CLS]', 'heart', 'melting', 'mum', '##ch', 'goblin', '##s',
       'eating', 'is', 'the', 'most', 'adorable', 'sight', 'ever', 'you',
       '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]'], dtype='<U8'), array(['[CLS]', 'this', 'sums', 'up', 'the', 'corona', '##virus',
       'literally', 'everyone', 'right', 'now', 'panic', 'fat', 'the',
       '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[

In [ ]:
def token_id(data):
  for index,i in enumerate(data):
    data[index]=np.array(tokenizer.convert_tokens_to_ids(i))
  return (data)

In [ ]:
train_tokens=token_id(train_tokens)
test_tokens=token_id(test_tokens)
validation_tokens=token_id(validation_tokens)

In [ ]:

train_mask=np.asarray(train_mask)
test_mask=np.asarray(test_mask)
validation_mask=np.asarray(validation_mask)

In [ ]:
train_tokens=np.asarray(train_tokens)
test_tokens=np.asarray(test_tokens)
validation_tokens=np.asarray(validation_tokens)

In [ ]:
# get the train output, BERT model will give one output so save in
# X_train_pooled_output
train_pooled_output=bert_model.predict([train_tokens,train_mask,train_segment])

In [ ]:
# get the test output, BERT model will give one output so save in
# test_pooled_output
test_pooled_output=bert_model.predict([test_tokens,test_mask,test_segment])
validation_pooled_output=bert_model.predict([validation_tokens,validation_mask,validation_segment])

In [ ]:
##save all your results to disk so that, no need to run all again. 
pickle.dump((train_pooled_output,validation_pooled_output, test_pooled_output),open('/content/drive/MyDrive/Meme/data/bert_output.pkl','wb'))

In [ ]:
##create an NN and 
def give_model():
  model=Sequential()
  model.add(Conv1D(256, 3,strides=5, activation='relu',input_shape=(768,1)))
  model.add(Conv1D(128, 3,strides=4, activation='relu'))
  model.add(Conv1D(64, 4,strides=2, activation='relu'))
  model.add(Conv1D(32, 2, activation='relu'))
  model.add(Flatten())

  
  model.add(Dense(128,activation='relu',kernel_initializer='he_normal'))
  model.add(Dropout(0.6))
  model.add(BatchNormalization())
  model.add(Dense(64,activation='relu',kernel_initializer='he_normal'))
  model.add(Dropout(0.6))
  model.add(BatchNormalization())

  model.add(Dense(32,activation='relu',kernel_initializer='he_normal'))
  model.add(Dropout(0.5))
  
  model.add(Dense(2,activation='softmax'))
  model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
  return model

In [ ]:
model =give_model()
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_12 (Conv1D)           (None, 154, 256)          1024      
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 38, 128)           98432     
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 18, 64)            32832     
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 17, 32)            4128      
_________________________________________________________________
flatten_3 (Flatten)          (None, 544)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               69760     
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)              

In [ ]:
y_train_one=tensorflow.keras.utils.to_categorical(train_data['dank_level'],2)
y_validation_one=tensorflow.keras.utils.to_categorical(validation_data['dank_level'],2)
y_test_one=tensorflow.keras.utils.to_categorical(test_data['dank_level'],2)

In [ ]:

train_pooled_output= train_pooled_output.reshape(train_pooled_output.shape[0], train_pooled_output.shape[1], 1)
validation_pooled_output= validation_pooled_output.reshape(validation_pooled_output.shape[0], validation_pooled_output.shape[1], 1)
test_pooled_output= test_pooled_output.reshape(test_pooled_output.shape[0], test_pooled_output.shape[1], 1)

print("Training shape---> ",train_pooled_output.shape)
print("Test shape    ---> ",test_pooled_output.shape)
print("validation shape    ---> ",validation_pooled_output.shape)


Training shape--->  (3264, 768, 1)
Test shape    --->  (1637, 768, 1)
validation shape    --->  (1615, 768, 1)


In [ ]:
model.fit(train_pooled_output,
           y_train_one,batch_size=128,
           validation_data=(validation_pooled_output,y_validation_one),
           epochs=100)

Epoch 1/100
26/26 [==============================] - 1s 21ms/step - loss: 1.4184 - accuracy: 0.5064 - val_loss: 0.6933 - val_accuracy: 0.4892
Epoch 2/100
26/26 [==============================] - 0s 12ms/step - loss: 1.3856 - accuracy: 0.5181 - val_loss: 0.6930 - val_accuracy: 0.5108
Epoch 3/100
26/26 [==============================] - 0s 12ms/step - loss: 1.3358 - accuracy: 0.5184 - val_loss: 0.6933 - val_accuracy: 0.4892
Epoch 4/100
26/26 [==============================] - 0s 11ms/step - loss: 1.3592 - accuracy: 0.4997 - val_loss: 0.6930 - val_accuracy: 0.5108
Epoch 5/100
26/26 [==============================] - 0s 11ms/step - loss: 1.3070 - accuracy: 0.5046 - val_loss: 0.6932 - val_accuracy: 0.4892
Epoch 6/100
26/26 [==============================] - 0s 11ms/step - loss: 1.2916 - accuracy: 0.4960 - val_loss: 0.6936 - val_accuracy: 0.4892
Epoch 7/100
26/26 [==============================] - 0s 11ms/step - loss: 1.2534 - accuracy: 0.5080 - val_loss: 0.6937 - val_accuracy: 0.4892
Epoch 